In [41]:
# Install uncommon packages (will supply requements.txt later)
#!pip install git+https://github.com/boudinfl/pke.git
#!pip install wordninja
#!pip install pyenchant
#!pip install --no-cache-dir pycorpora


import pandas as pd
import numpy as np
import os
import json
import wordninja
from os import listdir
import re
import sys
import ast
import pke
import enchant
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string
#import pycorpora
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
d = enchant.Dict("en_US") # ENGLISH Dictionary


# CHANGE THE PATH TO A DATASET HERE
basedir = os.path.join(os.sep, "media", "tie-server", "DATA", "Jens", "Crunchbase")


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tie-server/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/tie-server/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organization_descriptions.csv"), index_col="uuid")
#data.iloc[:,1].to_csv('CB_Export_17_08_07/descr.txt')
print("loaded descriptions")
data1 = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organizations.csv"), low_memory=False, index_col="uuid")
print("loaded org infos")

In [8]:
# Merge 2 csv files
# we can time to see which is more effective
#result = pd.concat([data, data1], axis=1, sort=False)
result1 = pd.merge(data, data1, how='inner', left_index=True, right_index=True)
print("merged")
# Delete all companies that do not have any descriptions
result1 = result1.dropna(subset=["description"])

merged


In [9]:
#print(((result.shape[0] - result1.shape[0])/result.shape[0]*100), "% of companies do not have any description")

In [10]:
result1.head()

,description,company_name,primary_role,permalink,domain,homepage_url,country_code,state_code,region,city,...,phone,facebook_url,linkedin_url,cb_url,logo_url,profile_image_url,twitter_url,alias,created_at,updated_at
uuid,,,,,,,,,,,,,,,,,,,,,
f5bb580f-d655-cf3f-9ade-eca5b3f2719f,"Android, Apple - iOS, Blackberry, Windows Phon...",VilarikA,company,/organization/vilarika,vilarika.com.br,http://vilarika.com.br/,BRA,NaN,Rio de Janeiro,Belo Horizonte,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/vilarika,https://www.crunchbase.com/organization/vilari...,http://public.crunchbase.com/t_api_images/v141...,NaN,NaN,2014-12-11 06:46:05,2016-09-07 00:03:51.67913
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,Formel D GmbH,company,/organization/formel-d-gmbh,formeld.com,http://www.formeld.com,DEU,NaN,DEU - Other,Troisdorf,...,+49 2241 9960,https://www.facebook.com/formeld,https://www.linkedin.com/company/formel-d-group,https://www.crunchbase.com/organization/formel...,https://www.crunchbase.com/organization/formel...,http://public.crunchbase.com/t_api_images/v148...,https://www.twitter.com/formeld_es,NaN,2016-06-01 06:58:37.692725,2017-07-18 07:22:19.15864
f5bd38d1-7719-2935-fb6c-defae39f5b93,[iNFoGooL](http://infogool.com) - The Informat...,infogool,company,/organization/infogool,infogool.com,http://infogool.com,NaN,NaN,NaN,NaN,...,8802393009,http://www.facebook.com/infogool,NaN,https://www.crunchbase.com/organization/infogool,https://www.crunchbase.com/organization/infogo...,http://public.crunchbase.com/t_api_images/v139...,https://www.twitter.com/infogool,NaN,2014-04-12 00:10:12,2016-09-08 22:02:05.585875
773adc18-132a-937d-8841-4f833e64dd56,The Jinfeng Gold Mine is an combined open pit ...,Jinfeng Mine,company,/organization/jinfeng-mine,NaN,NaN,CHN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/jinfen...,NaN,NaN,NaN,NaN,2016-12-07 06:43:06.955155,2016-12-07 06:45:09.14727
f5bdd48a-a09f-2f4c-bd71-4c4207c7731b,Peardoc offers online tools to convert HTML to...,Peardoc Solutions,company,/organization/peardoc-solutions,peardoc.com,http://www.peardoc.com,IND,NaN,Chennai,Chennai,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/peardo...,https://www.crunchbase.com/organization/peardo...,http://public.crunchbase.com/t_api_images/v140...,https://www.twitter.com/pear_doc,NaN,2014-07-17 10:06:03,2016-03-08 02:56:01.230586


In [11]:
len(result1)

353146

In [12]:
# Constuct a list of companies that have descriptions
d_companies_list = list(result1.loc[:, "company_name"])
d_companies_list = [company.lower() for company in d_companies_list]
#d_companies_list = [company for company in d_companies_list if all(letter.isalpha() for letter in company)]
#d_companies_list = [company for company in d_companies_list if not d.check(company)]
len(d_companies_list)

353146

In [13]:
# Construct a list of companies from dataset
companies_list = os.listdir(os.path.join(basedir, "News_1"))

len(companies_list)

20247

In [14]:
# Get rid of ".json" extension and apply uppercase
companies_list = [re.sub(".json", "", company) for company in companies_list]
print(len(companies_list))
# Get rid of companies with names containing anything else than latin letters (as well as containing "_" separating multiple words in names)
companies_list = [company.lower() for company in companies_list if all(letter in string.ascii_letters for letter in company)]
print(len(companies_list))
# Get rid of companies whose full names consist of one single word from english dictionary
companies_list = [company for company in companies_list if not d.check(company.lower())]
print(len(companies_list))
# Tries to separate a name string into multiple words and if succeeds gets rid of the company (a lot of false positives)
# companies_list = [company for company in companies_list if len(wordninja.split(company.lower())) < 2]
# print(len(companies_list))

20247
12608
11385


In [15]:
companies_list = [company for company in companies_list if company.lower() in d_companies_list]
len(companies_list)

10175

In [16]:
# Prepare the data frame of companies descriptions to be used in final data frame with articles
d_company_df = result1
d_company_df.loc[:,'company_name'] = d_company_df.loc[:,'company_name'].str.lower()
#d_company_df = d_company_df[d_company_df['company_name'].str.lower().isin(d_companies_list)]
#d_company_df = d_company_df[d_company_df['company_name'].isin(d_companies_list)]
d_company_df = d_company_df.reset_index().set_index(keys='company_name')#, verify_integrity=True)

# Delete companies outside or dataset of articles
d_company_df = d_company_df[d_company_df.index.isin(companies_list)]

# Delete the companies which have the same name
d_company_df = d_company_df[~d_company_df.index.duplicated(keep=False)]


d_company_df = d_company_df[['uuid', 'description']]
len(d_company_df)

9936

In [17]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
d_company_df.to_csv(os.path.join(basedir, 'temp_data', 'd_company_df.csv'), sep=";")
d_company_df.head()

,uuid,description
company_name,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur..."


In [18]:
### ONE WAY TO EXTRACT KEYWORDS ###

# With PKE Package
tmp = d_company_df

# initialize keyphrase extraction model, here TopicRank
import pke



import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tie-server/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
counter = 0
for index, row in d_company_df.iterrows():
    
    
    counter +=1
    #if counter >3:
    #    break
    
    extractor = pke.unsupervised.TopicRank()
    # load the content of the document, here document is expected to be in raw
    # format (i.e. a simple string of text) and preprocessing is carried out using nltk
    s = row["description"]
    #s = ' '.join(s.split())
    #s = "fwfewk w31231dmm! test test hello you piece of mind balls"
    extractor.read_text(s)

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives
    extractor.candidate_selection()

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    try:
        extractor.candidate_weighting()
        # N-best selection, keyphrases contains the 10 highest scored candidates as
        # (keyphrase, score) tuples
        keyphrases = extractor.get_n_best(n=10, stemming=False)
        keyphrases = sorted(keyphrases, key=lambda x: x[1], reverse=True)

        tmp.loc[index, 'kp_topicrank'] = str(keyphrases)
    
    except ValueError:
        print("Description for company " + index + 
              " (" + row['uuid'] + ")" +
              " is too short. Description: " + index)
        continue

        
        #if None in keyphrases:
        
    
tmp.tail()

Description for company ineomarketing (954b5316-3bcc-6081-d018-fec2c374d395) is too short. Description: ineomarketing
Description for company arrowsight (23bea55a-2e83-2b39-b53a-d8bd28d218e8) is too short. Description: arrowsight
Description for company looklive (6d3b1bed-302a-3331-6f43-87f91ca6a54b) is too short. Description: looklive
Description for company cityspark (f25f78b7-1a86-bf9d-e5da-24a538b81c61) is too short. Description: cityspark
Description for company rentabilities (aa8e1403-1db8-e5bf-6b5c-423bb826ccfd) is too short. Description: rentabilities
Description for company redkix (0510d73a-8709-bcae-5930-f047ee8b9db7) is too short. Description: redkix
Description for company shaser (538439c3-693e-0c54-76e3-f6c4c237370c) is too short. Description: shaser
Description for company bamboostr (83ccded4-e8ca-782f-2ef6-2c5b020fe48a) is too short. Description: bamboostr
Description for company metacert (83d3852a-f60d-2bac-24fe-afdbfb316fe9) is too short. Description: metacert
Descript

,uuid,description,kp_topicrank
company_name,,,
hooja,ff71401a-f5b0-1dd8-2b72-4a9e5d6ecba0,Hooja is a company focused on the mobile realm...,"[('service', 0.0785488995371759), ('hooja', 0...."
baseclick,d140620f-09bb-dd33-baaa-ecffff1d8bc6,baseclick GmbH is a start-up company funded an...,"[('technologies', 0.10249367027116733), ('star..."
biodetego,fef487a3-2505-e176-7bdc-d82db18525dc,"BioDetego is developing, VASPfore, a new bioma...","[('cancer treatment protocols', 0.105445975900..."
socialthreader,ffe12892-05c4-e8e4-d728-d9b54e7b3aeb,SocialThreader increases effectiveness of digi...,"[('campaign performance', 0.1546622347489285),..."
findally,cc693948-abf3-e3dc-f845-3e38ac718aab,Findally is an image-based product engine that...,"[('image-based product engine', 0.168320437934..."


In [25]:
k_company_df = tmp
len(k_company_df)

9936

In [26]:
k_company_df = k_company_df.dropna()
companies_list = k_company_df.index.get_values()
len(k_company_df)

9914

In [27]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
k_company_df.to_csv(os.path.join(basedir, 'temp_data', 'k_company_df.csv'), sep=";")
k_company_df.head()

,uuid,description,kp_topicrank
company_name,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[('market', 0.09826419107155268), ('technical ..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[('classâ€\x9d products', 0.09608849550420087)..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[('systems', 0.09053148884251386), ('developme..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[('thrombovision', 0.1093899921747511), ('medi..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[('access policies', 0.08053299604810367), ('f..."


In [28]:
# Get the list of paths to articles files and their sizes
paths = [(company, os.path.join(basedir, "News_1", "".join([company.upper(), ".json"]))) for company in companies_list]
paths = [path + (os.stat(path[1]).st_size,) for path in paths]
# Sort by sizes
paths = sorted(paths, key=lambda x: x[2], reverse=True)

print(paths[0:10])

[('brightline', '/media/tie-server/DATA/Jens/Crunchbase/News_1/BRIGHTLINE.json', 195149313), ('wherefor', '/media/tie-server/DATA/Jens/Crunchbase/News_1/WHEREFOR.json', 176922132), ('musx', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MUSX.json', 164413600), ('understory', '/media/tie-server/DATA/Jens/Crunchbase/News_1/UNDERSTORY.json', 158724779), ('crono', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CRONO.json', 110729460), ('gridco', '/media/tie-server/DATA/Jens/Crunchbase/News_1/GRIDCO.json', 107924262), ('lookback', '/media/tie-server/DATA/Jens/Crunchbase/News_1/LOOKBACK.json', 107008921), ('enervault', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ENERVAULT.json', 100682369), ('energysavvy', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ENERGYSAVVY.json', 99732850), ('humon', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HUMON.json', 95443584)]


In [60]:
all_articles = pd.DataFrame()
num_articles = 0
regex = re.compile('[%s]' % re.escape(string.punctuation))


In [ ]:
for index_f, file in enumerate(paths):
    #if index_f > 0:
    #    break
    
    print(file)
    print(index_f)

    try:
        articles = pd.read_json(file[1])
    except ValueError:
        print("empty/faulty file")
        continue

    if "content" not in articles.columns:
        print("no content")
        continue

    articles.loc[:, "company"] = file[0]
    articles.loc[:, "uuid"] = k_company_df.loc[file[0].lower(), "uuid"]
    articles.loc[:, "description"] = k_company_df.loc[file[0].lower(), "description"]

    articles.dropna(subset=["content"], inplace=True)

    texts = [regex.sub('', i).lower() for i in articles.loc[:, "content"]]
    keyphrases = ast.literal_eval(k_company_df.loc[file[0].lower(), "kp_topicrank"])
    num_articles = num_articles + len(texts)

    # Check whether the any of the keywords exist in the text
    bi = [1 
             if any([keyphrase in text 
                     for (keyphrase, prob) in (keyphrases)]) and 250 < len(str.split(text)) < 500 
             else 0
            for text in texts]

    articles.loc[:, "kp_topicrank_bi"] = bi
    articles.dropna(subset=["content"], inplace=True)

    all_articles = all_articles.append(articles, ignore_index=True)
    
    

('brightline', '/media/tie-server/DATA/Jens/Crunchbase/News_1/BRIGHTLINE.json', 195149313)
0
('wherefor', '/media/tie-server/DATA/Jens/Crunchbase/News_1/WHEREFOR.json', 176922132)
1
('musx', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MUSX.json', 164413600)
2
('understory', '/media/tie-server/DATA/Jens/Crunchbase/News_1/UNDERSTORY.json', 158724779)
3
('crono', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CRONO.json', 110729460)
4
('gridco', '/media/tie-server/DATA/Jens/Crunchbase/News_1/GRIDCO.json', 107924262)
5
('lookback', '/media/tie-server/DATA/Jens/Crunchbase/News_1/LOOKBACK.json', 107008921)
6
('enervault', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ENERVAULT.json', 100682369)
7
('energysavvy', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ENERGYSAVVY.json', 99732850)
8
('humon', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HUMON.json', 95443584)
9
('convo', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CONVO.json', 85078294)
10
('petrichor', '/media/tie-server/DATA

('coinbase', '/media/tie-server/DATA/Jens/Crunchbase/News_1/COINBASE.json', 34444440)
95
('tenksolar', '/media/tie-server/DATA/Jens/Crunchbase/News_1/TENKSOLAR.json', 34339822)
96
('xensource', '/media/tie-server/DATA/Jens/Crunchbase/News_1/XENSOURCE.json', 34272142)
97
('inquira', '/media/tie-server/DATA/Jens/Crunchbase/News_1/INQUIRA.json', 34177682)
98
('mirra', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MIRRA.json', 34171468)
99
('revaluate', '/media/tie-server/DATA/Jens/Crunchbase/News_1/REVALUATE.json', 34120621)
100
('solum', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SOLUM.json', 33896025)
101
('codexis', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CODEXIS.json', 33830727)
102
('playwith', '/media/tie-server/DATA/Jens/Crunchbase/News_1/PLAYWITH.json', 33678562)
103
('rdio', '/media/tie-server/DATA/Jens/Crunchbase/News_1/RDIO.json', 33520713)
104
('ophthotech', '/media/tie-server/DATA/Jens/Crunchbase/News_1/OPHTHOTECH.json', 33487483)
105
('geli', '/media/tie-server/D

('relypsa', '/media/tie-server/DATA/Jens/Crunchbase/News_1/RELYPSA.json', 24656498)
189
('hallux', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HALLUX.json', 24637636)
190
('theranos', '/media/tie-server/DATA/Jens/Crunchbase/News_1/THERANOS.json', 24611187)
191
('foodpanda', '/media/tie-server/DATA/Jens/Crunchbase/News_1/FOODPANDA.json', 24584576)
192
('carlock', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CARLOCK.json', 24515466)
193
('personali', '/media/tie-server/DATA/Jens/Crunchbase/News_1/PERSONALI.json', 24458012)
194
('xactly', '/media/tie-server/DATA/Jens/Crunchbase/News_1/XACTLY.json', 24406438)
195
('epcglobal', '/media/tie-server/DATA/Jens/Crunchbase/News_1/EPCGLOBAL.json', 24336772)
196
('mycroft', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MYCROFT.json', 24208551)
197
('ribbit', '/media/tie-server/DATA/Jens/Crunchbase/News_1/RIBBIT.json', 24131160)
198
('corevalve', '/media/tie-server/DATA/Jens/Crunchbase/News_1/COREVALVE.json', 24008753)
199
('shoppable', '/medi

('bazaarvoice', '/media/tie-server/DATA/Jens/Crunchbase/News_1/BAZAARVOICE.json', 18970800)
282
('adara', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ADARA.json', 18961335)
283
('mulesoft', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MULESOFT.json', 18943083)
284
('arkadin', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ARKADIN.json', 18928867)
285
('inogen', '/media/tie-server/DATA/Jens/Crunchbase/News_1/INOGEN.json', 18838708)
286
('jaspersoft', '/media/tie-server/DATA/Jens/Crunchbase/News_1/JASPERSOFT.json', 18718220)
287
('tracxn', '/media/tie-server/DATA/Jens/Crunchbase/News_1/TRACXN.json', 18594890)
288
('datto', '/media/tie-server/DATA/Jens/Crunchbase/News_1/DATTO.json', 18526624)
289
('strava', '/media/tie-server/DATA/Jens/Crunchbase/News_1/STRAVA.json', 18426358)
290
('inmediata', '/media/tie-server/DATA/Jens/Crunchbase/News_1/INMEDIATA.json', 18375625)
291
('mapd', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MAPD.json', 18354393)
292
('vringo', '/media/tie-server/DA

('otonomy', '/media/tie-server/DATA/Jens/Crunchbase/News_1/OTONOMY.json', 15564348)
375
('vovici', '/media/tie-server/DATA/Jens/Crunchbase/News_1/VOVICI.json', 15562862)
376
('mirantis', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MIRANTIS.json', 15536988)
377
('carto', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CARTO.json', 15475614)
378
('hautelook', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HAUTELOOK.json', 15437486)
379
('nanosys', '/media/tie-server/DATA/Jens/Crunchbase/News_1/NANOSYS.json', 15399602)
380
('eved', '/media/tie-server/DATA/Jens/Crunchbase/News_1/EVED.json', 15392742)
381
('drik', '/media/tie-server/DATA/Jens/Crunchbase/News_1/DRIK.json', 15386157)
382
('orthosensor', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ORTHOSENSOR.json', 15385836)
383
('speedconnect', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SPEEDCONNECT.json', 15361342)
384
('mophie', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MOPHIE.json', 15344198)
385
('dermira', '/media/tie-serv

...

('unitedlex', '/media/tie-server/DATA/Jens/Crunchbase/News_1/UNITEDLEX.json', 3240582)
1673
('mimir', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MIMIR.json', 3239473)
1674
('bridgevine', '/media/tie-server/DATA/Jens/Crunchbase/News_1/BRIDGEVINE.json', 3236375)
1675
('netstreams', '/media/tie-server/DATA/Jens/Crunchbase/News_1/NETSTREAMS.json', 3235589)
1676
('calyxo', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CALYXO.json', 3213612)
1677
('twofish', '/media/tie-server/DATA/Jens/Crunchbase/News_1/TWOFISH.json', 3213287)
1678
('cloudscaling', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CLOUDSCALING.json', 3211456)
1679
('primerevenue', '/media/tie-server/DATA/Jens/Crunchbase/News_1/PRIMEREVENUE.json', 3207980)
1680
('inivata', '/media/tie-server/DATA/Jens/Crunchbase/News_1/INIVATA.json', 3207384)
1681
('voltus', '/media/tie-server/DATA/Jens/Crunchbase/News_1/VOLTUS.json', 3206047)
1682
('sharethrough', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SHARETHROUGH.json', 3205239)

('zmanda', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ZMANDA.json', 3026901)
1765
('shoedazzle', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SHOEDAZZLE.json', 3024236)
1766
('hypr', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HYPR.json', 3019373)
1767
('surgiquest', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SURGIQUEST.json', 3016373)
1768
('urbanara', '/media/tie-server/DATA/Jens/Crunchbase/News_1/URBANARA.json', 3014940)
1769
('homejoy', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HOMEJOY.json', 3011448)
1770
('cuculus', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CUCULUS.json', 3011208)
1771
('howcast', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HOWCAST.json', 3008213)
1772
('scalix', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SCALIX.json', 3005350)
1773
('poshly', '/media/tie-server/DATA/Jens/Crunchbase/News_1/POSHLY.json', 2999115)
1774
('bunchball', '/media/tie-server/DATA/Jens/Crunchbase/News_1/BUNCHBALL.json', 2999030)
1775
('auctio', '/media/tie-ser

('dataminr', '/media/tie-server/DATA/Jens/Crunchbase/News_1/DATAMINR.json', 2820843)
1857
('depositphotos', '/media/tie-server/DATA/Jens/Crunchbase/News_1/DEPOSITPHOTOS.json', 2820446)
1858
('clearcare', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CLEARCARE.json', 2817487)
1859
('mojiva', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MOJIVA.json', 2816934)
1860
('insightly', '/media/tie-server/DATA/Jens/Crunchbase/News_1/INSIGHTLY.json', 2815564)
1861
('validic', '/media/tie-server/DATA/Jens/Crunchbase/News_1/VALIDIC.json', 2814819)
1862
('clarizen', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CLARIZEN.json', 2812437)
1863
('zoox', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ZOOX.json', 2812243)
1864
('gainsight', '/media/tie-server/DATA/Jens/Crunchbase/News_1/GAINSIGHT.json', 2809691)
1865
('mspot', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MSPOT.json', 2809169)
1866
('voltdb', '/media/tie-server/DATA/Jens/Crunchbase/News_1/VOLTDB.json', 2806806)
1867
('spaceclaim', '/m

('gainspeed', '/media/tie-server/DATA/Jens/Crunchbase/News_1/GAINSPEED.json', 2635183)
1949
('abine', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ABINE.json', 2632575)
1950
('healthrageous', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HEALTHRAGEOUS.json', 2630357)
1951
('mobisante', '/media/tie-server/DATA/Jens/Crunchbase/News_1/MOBISANTE.json', 2629065)
1952
('seatgeek', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SEATGEEK.json', 2626877)
1953
('protonet', '/media/tie-server/DATA/Jens/Crunchbase/News_1/PROTONET.json', 2625916)
1954
('enservio', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ENSERVIO.json', 2624317)
1955
('wiziq', '/media/tie-server/DATA/Jens/Crunchbase/News_1/WIZIQ.json', 2624092)
1956
('sinch', '/media/tie-server/DATA/Jens/Crunchbase/News_1/SINCH.json', 2622913)
1957
('newswhip', '/media/tie-server/DATA/Jens/Crunchbase/News_1/NEWSWHIP.json', 2622833)
1958
('nuzzel', '/media/tie-server/DATA/Jens/Crunchbase/News_1/NUZZEL.json', 2621527)
1959
('snapsheet', '/me

('globaltranz', '/media/tie-server/DATA/Jens/Crunchbase/News_1/GLOBALTRANZ.json', 2462523)
2041
('tyfone', '/media/tie-server/DATA/Jens/Crunchbase/News_1/TYFONE.json', 2461922)
2042
('stromedix', '/media/tie-server/DATA/Jens/Crunchbase/News_1/STROMEDIX.json', 2461337)
2043
('wegolook', '/media/tie-server/DATA/Jens/Crunchbase/News_1/WEGOLOOK.json', 2458300)
2044
('hopon', '/media/tie-server/DATA/Jens/Crunchbase/News_1/HOPON.json', 2458036)
2045
('controlscan', '/media/tie-server/DATA/Jens/Crunchbase/News_1/CONTROLSCAN.json', 2456028)
2046
('addepar', '/media/tie-server/DATA/Jens/Crunchbase/News_1/ADDEPAR.json', 2452113)
2047
('intelliworks', '/media/tie-server/DATA/Jens/Crunchbase/News_1/INTELLIWORKS.json', 2450367)
2048
('pertino', '/media/tie-server/DATA/Jens/Crunchbase/News_1/PERTINO.json', 2449691)
2049
('embedly', '/media/tie-server/DATA/Jens/Crunchbase/News_1/EMBEDLY.json', 2449447)
2050
('dynamicops', '/media/tie-server/DATA/Jens/Crunchbase/News_1/DYNAMICOPS.json', 2446916)
2051


In [59]:
all_articles["kp_topicrank_bi"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
       ... 
5662    0.0
5663    0.0
5664    0.0
5665    1.0
5666    1.0
5667    0.0
5668    0.0
5669    0.0
5670    0.0
5671    0.0
5672    0.0
5673    0.0
5674    0.0
5675    0.0
5676    0.0
5677    0.0
5678    0.0
5679    0.0
5680    0.0
5681    0.0
5682    0.0
5683    0.0
5684    0.0
5685    0.0
5686    0.0
5687    0.0
5688    0.0
5689    1.0
5690    1.0
5691    0.0
Name: kp_topicrank_bi, Length: 5692, dtype: float64

In [148]:
print(str(len(all_articles/num_articles)) + "% of articles were are containing corresponding keyphrases extracted from the descriptions of the companies")

0% of articles were are containing corresponding keyphrases extracted from the descriptions of the companies


In [ ]:
# Save dataset
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
all_articles.to_csv(os.path.join(basedir, 'temp_data', 'all_articles.csv'), sep=";")
all_articles.head()

In [20]:
print(companies_list)

['ABRA', 'ACRONIS', 'ACTON', 'AIRSTONE', 'ALBERT', 'ALEA', 'ALIANZA', 'AMARA', 'ANDO', 'ARKIN', 'ARTSPACE', 'ASANA', 'ASOKA', 'ASTRID', 'ATHOS', 'AVA', 'AYR', 'BACKDOOR', 'BANNERMAN', 'BARKLY', 'BASTILLE', 'BBB', 'BEBO', 'BENTO', 'BIA', 'BINDO', 'BIOME', 'BIRDBOX', 'BITTORRENT', 'BIX', 'BIZEN', 'BOARDVOTE', 'BONOBOS', 'BOTTLENOSE', 'BOWERY', 'BRAINTREE', 'BRIT', 'BUDDYTV', 'CAIS', 'CALERA', 'CALVIN', 'CARTESIAN', 'CASPER', 'CASSATT', 'CENTRO', 'CERBERUS', 'CHAI', 'CLARK', 'CLEO', 'CLICKABLE', 'CLOE', 'COTOPAXI', 'CRONO', 'CROSSFADER', 'DAILYMOTION', 'DELOS', 'DIGBY', 'DIY', 'DOO', 'DOPPELGANGER', 'ECO', 'ELASTICA', 'ELEMENTUM', 'ELLIE', 'ELɘ', 'EMMA', 'ENDORPHIN', 'EPIPHYTE', 'EPS', 'ESS', 'EVERLY', 'EXCALIBUR', 'EXOGENESIS', 'EYESPOT', 'FAB', 'FACEBOOK', 'FANBASE', 'FARADAY', 'FIDELIS', 'FLASHPOINT', 'FLAVOUR', 'FLYER', 'FLYNN', 'FONDU', 'FORMSPRING', 'FRONTO', 'FRS', 'FULHAM', 'FUZE', 'FY', 'GAMETIME', 'GAMEZONE', 'GAMGEE', 'GELI', 'GENI', 'GOLDSTAR', 'GOLGI', 'GORGIAS', 'GRO', 'HALL

In [56]:
len(articles)

76920

In [175]:
all_articles.tail()

,UNPARSED,autor,byline,contact,content,dateline,distribution,error,error_code,grafic,...,pub_type,rubrik,section,series,source,company,uuid,description,country,correction_date
4395,"AFX - Asia\n\n November 2,...",NaN,NaN,NaN,"AFX - Asia\n\n November 2,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4396,"AFX - Asia\n\n August 16,...",NaN,NaN,NaN,"AFX - Asia\n\n August 16,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4397,Manila Times\n\n J...,NaN,NaN,NaN,Manila Times\n\n J...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4398,"AFX - Asia\n\n May 24, 2...",NaN,NaN,NaN,"AFX - Asia\n\n May 24, 2...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4399,"AFX - Asia\n\n April 22,...",NaN,NaN,NaN,"AFX - Asia\n\n April 22,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
